In [1]:
import numpy as np

In [2]:
class Value:
    def __init__(self, data, prev=(), op=None):
        self.data = data
        self._prev = prev
        self._op = op
        self.grad = 0.0

    def __repr__(self):
        return f"Value(data={self.data})"

    def __str__(self,):
        return f"{self.data}"
    
    def __add__(self, other):
        if isinstance(other, (int, float)):
            other = Value(other)
        return Value(self.data + other.data, prev=(self, other), op='+')
    
    def __radd__(self, other):
        return self.__add__(other)

    def __mul__(self, other):
        if isinstance(other, (int, float)):
            other = Value(other)
        return Value(self.data * other.data, prev=(self, other), op='*')
        
    def __rmul__(self, other):
        return self.__mul__(other)
    
    def __gt__(self, other):
        if isinstance(other, (int, float)):
            other = Value(other)
        return self.data > other.data

    def ReLu(self,):
        return Value(max(0, self.data), prev=(self,), op='ReLu')

    @staticmethod
    def backpropagation(y):
        y.grad = 1
        Value._propagate(y)

    @staticmethod
    def _propagate(y):
        prev = y._prev

        if prev is None:
            return
        
        if len(prev) == 1:
            p = prev[0]
            if y._op == 'ReLu':
                dp = 1 if p > 0 else 0
                p.grad += y.grad * dp
                Value._propagate(p)
        
        if len(prev) == 2:
            p1, p2 = prev[0], prev[1]
            if y._op == '+':
                dp1 = 1
                dp2 = 1
            if y._op == '*':
                dp1 = p2.data
                dp2 = p1.data 
            p1.grad += y.grad * dp1
            p2.grad += y.grad * dp2
            Value._propagate(p1)
            Value._propagate(p2)

In [3]:
a = Value(4)
b = Value(2)

c = 5 + a
d = 2 + b
e = c * d
f = b * e
g = f * a
h = 3 * g

In [4]:
Value.backpropagation(h)

In [5]:
a.grad

312.0

In [6]:
class Neuron:
    def __init__(self, weights, bias):
        self.weights = [Value(weight) for weight in weights]
        self.bias = Value(bias)

    def forward(self, x):
        if isinstance(x, list):
            x = np.array(x)
        return (x @ self.weights + self.bias).ReLu()

In [7]:
class Layer:
    def __init__(self, neurons):
        self.neurons = neurons

    def forward(self, x):
        if isinstance(x, list):
            x = np.array(x)
        return np.array([n.forward(x) for n in self.neurons])

In [8]:
class MLP:
    def __init__(self, layers):
        self.layers = layers
    
    def forward(self, x):
        if isinstance(x, list):
            x = np.array(x)
        for l in self.layers:
            x = l.forward(x)
        return x

In [9]:
n1 = Neuron([0.3, 0.2, 0.5], 1.2)
n2 = Neuron([0.4, 0.1, 0.6], 0.2)
n3 = Neuron([0.6, 0.2, 0.5], -1.0)

In [10]:
l1 = Layer([n1, n2, n3])

In [11]:
n4 = Neuron([0.3, 0.3, 0.4], 0.0)
n5 = Neuron([0.1, 0.2, 0.8], 0.3)
n6 = Neuron([0.5, 0.1, 0.5], -1.8)

In [12]:
l2 = Layer([n4, n5])

In [13]:
mlp = MLP([l1, l2, Layer([Neuron([1.3, 3.3], 0.1)])])

In [14]:
x = [1.2, 6.8, 0.3]

In [15]:
mlp.forward(x)

array([Value(data=8.8042)], dtype=object)